# Assignment 1: Extracting text from PDF

In [1]:
# Import all necessary Libraries

import fitz  # PyMuPDF
import cv2
import numpy as np
import openpyxl
from openpyxl.styles import Alignment
import pandas as pd

In [2]:
# Python Script for extracting text from pdf using PYMuPDF Lib

def extract_text_blocks(pdf_path, page_number):
    # Open the PDF document using PyMuPDF
    doc = fitz.open(pdf_path)
    text_blocks = []

    # Extract text blocks from the specified page
    page = doc[page_number]
    blocks = page.get_text("blocks")

    # Collect text, x, and y coordinates for each block
    for block in blocks:
        text = block[4]
        x, y, _, _ = block[:4]
        text_blocks.append((x, y, text))

    # Sort text blocks based on coordinate
    return sorted(text_blocks, key=lambda x: (x[0], x[1]))

def group_blocks_by_columns(text_blocks, column_threshold=50, vertical_threshold=30):
    grouped_blocks = []
    current_column = []
    prev_x = None
    prev_y = None

    # Iterate through each text block
    for x, y, text in text_blocks:
        # Check if the block belongs to the current column based on both x and y coordinates
        if (prev_x is None or abs(x - prev_x) < column_threshold) and (prev_y is None or abs(y - prev_y) < vertical_threshold):
            current_column.append(text)
        else:
            # Start a new column if x-coordinate difference or y-coordinate difference exceeds the threshold
            grouped_blocks.append(current_column)
            current_column = [text]

        prev_x = x
        prev_y = y

    # Add the last column
    grouped_blocks.append(current_column)
    return grouped_blocks

def write_to_excel(grouped_blocks, excel_path):
    # Create a new Excel workbook and select the active sheet
    wb = openpyxl.Workbook()
    ws = wb.active

    # Write each text block in a new row
    for column in grouped_blocks:
        for text_block in column:
            ws.append([text_block])

    # Adjust column width based on the maximum length of text in the column
    max_length = max(len(text_block) for column in grouped_blocks for text_block in column)
    adjusted_width = max_length + 2
    ws.column_dimensions['A'].width = adjusted_width

    # Set text alignment to wrap for each cell
    for row in ws.iter_rows():
        for cell in row:
            cell.alignment = Alignment(wrap_text=True)

    # Save the Excel workbook
    wb.save(excel_path)


def main(pdf_path, excel_path, page_number):
    # Extract text blocks from the specified page of the PDF document
    text_blocks = extract_text_blocks(pdf_path, page_number)
    
    # Group text blocks into columns based on x-coordinate differences
    grouped_blocks = group_blocks_by_columns(text_blocks)
    
    # Write the grouped text blocks to an Excel file
    write_to_excel(grouped_blocks, excel_path)

if __name__ == "__main__":
    # Specify the path to the input PDF document, the output Excel file, and the page number
    pdf_path = "C:/Users/stuti/anaconda3/envs/psupr/WalnutAI/keppel-corporation-limited-annual-report-2018.pdf"
    excel_path = "output1.xlsx"
    page_number = 11  # Replace with the desired page number (starting from 0)

    # Run the main function
    task1_excel= main(pdf_path, excel_path, page_number)

Once this python script sucessfully runs, it dumps the extracted text to output1.xlsx

# Assignment 2: Extracting table from PDF

Initially, plan was to use Cascadetabnet (A pre-trained DNN for table detection). However, due to compatibility/version issue I wasn't able to run in both anaconda and google colab. Thus, I only proceeded using Camelot library 

In [3]:
import camelot

def extract_tables_from_page(pdf_path, page_number):
    # Specify the path to PDF file
    pdf_path = "C:/Users/stuti/anaconda3/envs/psupr/WalnutAI/keppel-corporation-limited-annual-report-2018.pdf"

    # Use camelot to extract tables from a specific page
    tables = camelot.read_pdf(pdf_path, flavor='stream', pages=str(page_number))

    # Access each table
    for i, table in enumerate(tables):
        df = table.df
        # Now 'df' contains the extracted table as a pandas DataFrame
        # Save it to Excel 
        df.to_excel(f"table_page_{page_number}_{i}.xlsx", index=False)

# Specify the page number you want to extract tables from
page_number = 69  # Change this to desired page number (starting from 1)
pdf_path = "C:/Users/stuti/anaconda3/envs/psupr/WalnutAI/keppel-corporation-limited-annual-report-2018.pdf"

# Call the function to extract tables from the specified page
extract_tables_from_page(pdf_path, page_number)
